In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from collections import deque
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
last5_img = deque(maxlen=5)
last5_keyout = deque(maxlen=5)
data_mean = 119.55953717374616
data_std = 77.89011500152353

In [2]:
X = tf.placeholder(tf.float32, [5, 200, 400, 3])
Y = tf.placeholder(tf.float32, [5, hidden_size])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv2 = tf.layers.conv2d(inputs=X, filters=128, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    
    pool3 = tf.reshape(pool, [1, 5, -1])

In [3]:
with tf.variable_scope("rnn"):
    cell = tf.contrib.rnn.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    #outputs : 1, 5, 50*99*16
    outputs, _states = tf.nn.dynamic_rnn(cell, pool3, initial_state=initial_state, dtype=tf.float32)
    
    X_for_sigmoid = tf.reshape(outputs, [-1, hidden_size])
    
    sigmoid_w = tf.get_variable("softmax_w", [hidden_size, hidden_size])
    sigmoid_b = tf.get_variable("softmax_b", [hidden_size])
    outputs = tf.sigmoid(tf.matmul(X_for_sigmoid, sigmoid_w) + sigmoid_b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [4]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(1):

            data_df = pd.read_csv('log.csv', names=['name','key_out'])
            names = data_df['name'].values
            name_i = 0
            # keyouts = [8, 1, 4, 2, 15, ...]
            key_out = [eval(x) for x in data_df['key_out'].values]

            while len(last5_img) < 4 and len(names)>4:
                img = Image.open("./img_rgb/"+names[name_i])
                data = np.array( img, dtype='uint8' )
                data = (data-data_mean) / data_std
                last5_img.append(data)
                last5_keyout.append(key_out[name_i])
                name_i+=1

            while(name_i < len(names)):
                img = Image.open("./img_rgb/"+names[name_i])
                data = np.array( img, dtype='uint8')
                last5_img.append(data)
                x_data = np.reshape(np.array(last5_img), [5, 200, 400, 3])
                last5_keyout.append(key_out[name_i])

                print name_i, "/", len(names), sess.run(loss, feed_dict={X:x_data, Y:last5_keyout})
                sess.run(train, feed_dict={X:np.array(x_data, np.float32), Y:last5_keyout})
                name_i += 1
            print 'epoch', epoch+1
    finally:
        save_path = saver.save(sess, "./model/"+model_name+".ckpt")

Do you want addtional learning? (Name or Enter)
Model_Name: v6_1
4 / 1954 0.6424315
5 / 1954 0.5334995
6 / 1954 0.5217999
7 / 1954 0.5207984
8 / 1954 0.51467395
9 / 1954 0.532104
10 / 1954 0.52608514
11 / 1954 0.54438365
12 / 1954 0.5442725
13 / 1954 0.54416025
14 / 1954 0.544047
15 / 1954 0.543933
16 / 1954 0.54381835
17 / 1954 0.6098737
18 / 1954 0.5938066
19 / 1954 0.5776471
20 / 1954 0.64275277
21 / 1954 0.7023155
22 / 1954 0.65609586
23 / 1954 0.7350646
24 / 1954 0.7146762
25 / 1954 0.6338237
26 / 1954 0.64234316
27 / 1954 0.63567895
28 / 1954 0.65567696
29 / 1954 0.66850233
30 / 1954 0.73121357
31 / 1954 0.6574874
32 / 1954 0.64908427
33 / 1954 0.59140015
34 / 1954 0.6329843
35 / 1954 0.56986135
36 / 1954 0.55898535
37 / 1954 0.54318273
38 / 1954 0.6091866
39 / 1954 0.54673934
40 / 1954 0.5463223
41 / 1954 0.54350096
42 / 1954 0.5251905
43 / 1954 0.44685364
44 / 1954 0.46477085
45 / 1954 0.5464269
46 / 1954 0.6272508
47 / 1954 0.6365054
48 / 1954 0.73317146
49 / 1954 0.7946865
50

390 / 1954 0.7594254
391 / 1954 0.68161666
392 / 1954 0.5980488
393 / 1954 0.5985967
394 / 1954 0.6026839
395 / 1954 0.690302
396 / 1954 0.75815046
397 / 1954 0.7645034
398 / 1954 0.6993867
399 / 1954 0.6219796
400 / 1954 0.55836403
401 / 1954 0.56790316
402 / 1954 0.56915015
403 / 1954 0.5776609
404 / 1954 0.57847977
405 / 1954 0.57039195
406 / 1954 0.514809
407 / 1954 0.5147204
408 / 1954 0.5089542
409 / 1954 0.5737472
410 / 1954 0.56117404
411 / 1954 0.5543054
412 / 1954 0.6113721
413 / 1954 0.6180066
414 / 1954 0.63488466
415 / 1954 0.6937014
416 / 1954 0.76377666
417 / 1954 0.7716515
418 / 1954 0.82696897
419 / 1954 0.74981034
420 / 1954 0.68491644
421 / 1954 0.68518424
422 / 1954 0.6108975
423 / 1954 0.56401265
424 / 1954 0.5647528
425 / 1954 0.55803776
426 / 1954 0.51132405
427 / 1954 0.5130589
428 / 1954 0.5009416
429 / 1954 0.5008934
430 / 1954 0.4890011
431 / 1954 0.49029678
432 / 1954 0.49910766
433 / 1954 0.5021058
434 / 1954 0.5107144
435 / 1954 0.5123432
436 / 1954 0.5122

773 / 1954 0.47062245
774 / 1954 0.47982112
775 / 1954 0.55275923
776 / 1954 0.5432704
777 / 1954 0.6078318
778 / 1954 0.6719497
779 / 1954 0.73332584
780 / 1954 0.73514193
781 / 1954 0.71629715
782 / 1954 0.6448799
783 / 1954 0.63952875
784 / 1954 0.57875454
785 / 1954 0.5298873
786 / 1954 0.53548896
787 / 1954 0.598653
788 / 1954 0.5474426
789 / 1954 0.5372623
790 / 1954 0.5448388
791 / 1954 0.6002835
792 / 1954 0.5375575
793 / 1954 0.54629165
794 / 1954 0.60957235
795 / 1954 0.66608655
796 / 1954 0.67517054
797 / 1954 0.7382166
798 / 1954 0.79329056
799 / 1954 0.7286442
800 / 1954 0.664042
801 / 1954 0.5996346
802 / 1954 0.5364719
803 / 1954 0.4812693
804 / 1954 0.4811941
805 / 1954 0.48111543
806 / 1954 0.48103362
807 / 1954 0.48094898
808 / 1954 0.48086175
809 / 1954 0.48077226
810 / 1954 0.4806808
811 / 1954 0.47397742
812 / 1954 0.5383562
813 / 1954 0.53842735
814 / 1954 0.5394061
815 / 1954 0.54607475
816 / 1954 0.59967387
817 / 1954 0.5444614
818 / 1954 0.5442046
819 / 1954 0.

1149 / 1954 0.5654595
1150 / 1954 0.50195444
1151 / 1954 0.4572846
1152 / 1954 0.5233546
1153 / 1954 0.531165
1154 / 1954 0.59570324
1155 / 1954 0.58893806
1156 / 1954 0.57980764
1157 / 1954 0.5169978
1158 / 1954 0.5142145
1159 / 1954 0.45715714
1160 / 1954 0.45712394
1161 / 1954 0.45374098
1162 / 1954 0.45487046
1163 / 1954 0.46252146
1164 / 1954 0.4547476
1165 / 1954 0.45880705
1166 / 1954 0.4535079
1167 / 1954 0.5172887
1168 / 1954 0.5259081
1169 / 1954 0.5277812
1170 / 1954 0.5883652
1171 / 1954 0.5805954
1172 / 1954 0.5267106
1173 / 1954 0.52901995
1174 / 1954 0.51316226
1175 / 1954 0.46418896
1176 / 1954 0.4595316
1177 / 1954 0.45138472
1178 / 1954 0.4562122
1179 / 1954 0.51985425
1180 / 1954 0.5898491
1181 / 1954 0.5807565
1182 / 1954 0.5793814
1183 / 1954 0.5706256
1184 / 1954 0.510517
1185 / 1954 0.5270068
1186 / 1954 0.5213118
1187 / 1954 0.52107406
1188 / 1954 0.5198196
1189 / 1954 0.57462406
1190 / 1954 0.5178207
1191 / 1954 0.51445
1192 / 1954 0.58098954
1193 / 1954 0.6370

1515 / 1954 0.4414146
1516 / 1954 0.4413746
1517 / 1954 0.45079207
1518 / 1954 0.4518264
1519 / 1954 0.45881385
1520 / 1954 0.45000038
1521 / 1954 0.4412045
1522 / 1954 0.44202918
1523 / 1954 0.44195008
1524 / 1954 0.44186848
1525 / 1954 0.5023802
1526 / 1954 0.50228035
1527 / 1954 0.5010706
1528 / 1954 0.4998907
1529 / 1954 0.4916196
1530 / 1954 0.43987298
1531 / 1954 0.5078276
1532 / 1954 0.50145805
1533 / 1954 0.5701869
1534 / 1954 0.5685967
1535 / 1954 0.55337733
1536 / 1954 0.49939328
1537 / 1954 0.49193543
1538 / 1954 0.44973487
1539 / 1954 0.44992638
1540 / 1954 0.45691115
1541 / 1954 0.4577028
1542 / 1954 0.44997287
1543 / 1954 0.518246
1544 / 1954 0.50917685
1545 / 1954 0.56086683
1546 / 1954 0.62095505
1547 / 1954 0.6739692
1548 / 1954 0.6813207
1549 / 1954 0.73317325
1550 / 1954 0.6724075
1551 / 1954 0.6723298
1552 / 1954 0.6110495
1553 / 1954 0.5515135
1554 / 1954 0.49790972
1555 / 1954 0.4909237
1556 / 1954 0.4457116
1557 / 1954 0.43853226
1558 / 1954 0.4463145
1559 / 1954

1879 / 1954 0.42560142
1880 / 1954 0.4849183
1881 / 1954 0.4989253
1882 / 1954 0.5127962
1883 / 1954 0.52553236
1884 / 1954 0.5759813
1885 / 1954 0.51403415
1886 / 1954 0.5593209
1887 / 1954 0.5456447
1888 / 1954 0.5350607
1889 / 1954 0.48300838
1890 / 1954 0.47568697
1891 / 1954 0.42568278
1892 / 1954 0.42660484
1893 / 1954 0.4277051
1894 / 1954 0.4298951
1895 / 1954 0.43904155
1896 / 1954 0.49744
1897 / 1954 0.4963603
1898 / 1954 0.5544218
1899 / 1954 0.5510004
1900 / 1954 0.53435564
1901 / 1954 0.4976243
1902 / 1954 0.5046347
1903 / 1954 0.5130753
1904 / 1954 0.52784234
1905 / 1954 0.5248276
1906 / 1954 0.56121695
1907 / 1954 0.55148935
1908 / 1954 0.50010645
1909 / 1954 0.494625
1910 / 1954 0.47932944
1911 / 1954 0.43762332
1912 / 1954 0.4374447
1913 / 1954 0.524325
1914 / 1954 0.516246
1915 / 1954 0.52513754
1916 / 1954 0.5227931
1917 / 1954 0.59558475
1918 / 1954 0.5161214
1919 / 1954 0.5834023
1920 / 1954 0.5944464
1921 / 1954 0.5788968
1922 / 1954 0.498033
1923 / 1954 0.490523
